# Forecasting Using Data Stored In Apache Druid

## 1. Load the data from Druid

In [1]:
import json
import requests
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler

In [2]:
# # define the Druid URL
# url = 'http://34.255.123.172:8888/druid/v2/?pretty'

# # define the Druid query
# query = {'queryType': 'scan', 
#          'dataSource': '2_1_OrdersFlaviaNew', 
#          'intervals': ['2020-06-01T08:00:00.000Z/2020-06-01T10:00:00.000Z'],
#          'granularity': 'all'}

# # run the Druid query
# results = json.dumps(requests.post(url, headers={'Content-Type': 'application/json'}, json=query).json()[1]['events'])

# # organize the results of the Druid 
# # query in a pandas data frame
# df = pd.read_json(results, orient='records')
# df = df[['__time', 'Value']]
# df.rename(columns={'__time': 'time', 'Value': 'value'}, inplace=True)
# df.sort_values(by='time', inplace=True)


# the following 3 lines will need to be deleted
df = pd.read_csv('data.csv', index_col=0)
df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%d %H:%M:%S')

For computational efficiency reasons, we aggregate the data by minute.

In [3]:
df.index = pd.DatetimeIndex(df['time'])
df.drop(labels='time', axis=1, inplace=True)
df = df.resample('T').sum()

Print the first few rows of the data frame.

In [4]:
df.head()

,value
time,
2020-06-01 00:00:00,4471.638465
2020-06-01 00:01:00,4480.949182
2020-06-01 00:02:00,4571.854005
2020-06-01 00:03:00,4533.082060
2020-06-01 00:04:00,4570.536454


Print the last few rows of the data frame.

In [5]:
df.tail()

,value
time,
2020-06-07 23:55:00,4519.431081
2020-06-07 23:56:00,4536.116809
2020-06-07 23:57:00,4613.311237
2020-06-07 23:58:00,4391.271856
2020-06-07 23:59:00,4560.330629


## 2. Perform some basic data analysis using pandas

Calculate the descriptive statistics.

In [6]:
df.describe()

,value
count,10080.000000
mean,5999.968054
std,1064.532300
min,4290.550730
25%,4943.955844
50%,6000.920431
75%,7056.443828
max,7770.848421


## 3. Forecast with the XGBoost model

Define the length of the forecasting window.

In [7]:
n = 24 * 60

Construct the features.

In [8]:
t_past = df.index
X_past = np.transpose(np.vstack([t_past.day.values, t_past.hour.values, t_past.minute.values]))

Scale the features.

In [9]:
scaler = MinMaxScaler((-1, 1)).fit(X_past)
X_past = scaler.transform(X_past)

Fit the model.

In [10]:
xgboost = xgb.XGBRegressor().fit(X_past, df['value'].values)

[08:24:28] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Examine the feature importances.

In [11]:
[[x, format(y, '.4f')] for x, y in zip(['day', 'hour', 'minute'], xgboost.feature_importances_.tolist())]

[['day', '0.0001'], ['hour', '0.9921'], ['minute', '0.0078']]

Extract the in-sample predictions.

In [12]:
predictions = pd.DataFrame({'time': t_past, 'value': xgboost.predict(X_past)})
predictions.head()

,time,value
0,2020-06-01 00:00:00,4486.040527
1,2020-06-01 00:01:00,4486.040527
2,2020-06-01 00:02:00,4487.715332
3,2020-06-01 00:03:00,4487.715332
4,2020-06-01 00:04:00,4493.330566


Generate the out-of-sample forecasts.

In [13]:
t_future = pd.date_range(start=df.index[-1], periods=n, freq='T')
X_future = np.transpose(np.vstack([t_future.day.values, t_future.hour.values, t_future.minute.values]))
X_future = scaler.transform(X_future)

In [14]:
forecasts = pd.DataFrame({'time': t_future, 'value': xgboost.predict(X_future)})
forecasts.head()

,time,value
0,2020-06-07 23:59:00,4482.617676
1,2020-06-08 00:00:00,4489.125000
2,2020-06-08 00:01:00,4489.125000
3,2020-06-08 00:02:00,4490.799805
4,2020-06-08 00:03:00,4490.799805


## 4. Visualize the results with Plotly

In [21]:
# create the layout
layout = {'plot_bgcolor': 'white',
          'paper_bgcolor': 'white',
          'margin': {'t':10, 'b':10, 'l':10, 'r':10, 'pad':0},
          'legend': {'x': 0, 'y': 1.1, 'orientation': 'h'},
          'font': {'size': 9},
          'yaxis': {'showgrid': True,
                    'zeroline': False,
                    'mirror': True,
                    'color': '#737373',
                    'linecolor': '#d9d9d9',
                    'gridcolor': '#d9d9d9',
                    'tickformat': '$,.0f'},
          'xaxis': {'showgrid': True,
                    'zeroline': False,
                    'mirror': True,
                    'color': '#737373',
                    'linecolor': '#d9d9d9',
                    'gridcolor': '#d9d9d9',
                    'type': 'date',
                    'tickformat': '%d %b %y %H:%M',
                    'tickangle': 0,
                    'nticks': 5}}

# create the traces
data = []

data.append(go.Scatter(x=df.index,
                       y=df['value'],
                       mode='markers',
                       marker=dict(color='#343a40', size=4),
                       name='Actual',
                       hovertemplate='<b>Actual</b><br>'
                       '<b>Time:</b> %{x|%d %b %Y %H:%M}<br>'
                       '<b>Value:</b> %{y: $,.2f}<extra></extra>'))

data.append(go.Scatter(x=predictions['time'],
                       y=predictions['value'],
                       mode='lines',
                       line=dict(color='#e83e8c', width=2),
                       name='In-Sample Predictions',
                       hovertemplate='<b>In-Sample Predictions</b></br>'
                       '<b>Time:</b> %{x|%d %b %Y %H:%M}<br>'
                       '<b>Value:</b> %{y: $,.2f}<extra></extra>'))

data.append(go.Scatter(x=forecasts['time'],
                       y=forecasts['value'],
                       mode='lines',
                       line=dict(color='#8A348E', width=2, dash='dash'),
                       name='Out-Of-Sample Forecasts',
                       hovertemplate='<b>Out-Of-Sample Forecasts</b></br>'
                       '<b>Time:</b> %{x|%d %b %Y %H:%M}<br>'
                       '<b>Value:</b> %{y: $,.2f}<extra></extra>'))

# create the figure
fig = go.Figure(data=data, layout=layout)

# display the figure
fig.show()

# save the plot
fig.write_html('forecasts_plot.html')